In [ ]:
# Import Qiskit IBM Runtime Service
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler

# Import Qiskit modules
from qiskit import Aer, QuantumCircuit, transpile, assemble, execute
from qiskit.utils import QuantumInstance
from qiskit.algorithms import VQE, Shor, Grover, AmplificationProblem, QAOA
from qiskit.algorithms.optimizers import COBYLA
from qiskit.circuit.library import TwoLocal, QFT, PhaseOracle
from qiskit_nature.drivers import PySCFDriver
from qiskit_nature.transformers import FreezeCoreTransformer
from qiskit_nature.converters import QubitConverter
from qiskit_nature.mappers import ParityMapper
from qiskit_nature.algorithms import GroundStateEigensolver
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.algorithms import MinimumEigenOptimizer

In [3]:
from qiskit_ibm_runtime import QiskitRuntimeService

# Save account to disk and save it as the default, with overwrite set to True.
QiskitRuntimeService.save_account(
    channel="ibm_cloud",
    token="API_TOKEN",
    instance="CRN",
    name="account-name",
    set_as_default=True,
    overwrite=True
)

# Load the saved credentials
service = QiskitRuntimeService(name="account-name")

In [5]:
from qiskit import QuantumCircuit
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler

# Create empty circuit
example_circuit = QuantumCircuit(2)
example_circuit.measure_all()

# Initialize QiskitRuntimeService
service = QiskitRuntimeService()

# List available backends
backends = service.backends()
for backend in backends:
    print(backend)

# Specify a known backend
backend_name = "ibmq_qasm_simulator"  # Replace with an available backend name
backend = service.backend(backend_name)

sampler = Sampler(backend)
job = sampler.run([example_circuit])
print(f"job id: {job.job_id()}")
result = job.result()
print(result)

<IBMBackend('ibmq_qasm_simulator')>
<IBMBackend('simulator_stabilizer')>
<IBMBackend('simulator_mps')>
<IBMBackend('simulator_extended_stabilizer')>
<IBMBackend('simulator_statevector')>
job id: csbssp97ufh2pu1lqpgg
PrimitiveResult([SamplerPubResult(data=DataBin(meas=BitArray(<shape=(), num_shots=4096, num_bits=2>)), metadata={'circuit_metadata': {}})], metadata={'execution': {'execution_spans': ExecutionSpans([SliceSpan(<start='2024-10-22 16:10:15', stop='2024-10-22 16:10:15', size=4096>)])}, 'version': 2})


In [8]:
from qiskit_ibm_runtime import QiskitRuntimeService

service = QiskitRuntimeService()
backends = service.backends()
for backend in backends:
    print(backend)

<IBMBackend('ibmq_qasm_simulator')>
<IBMBackend('simulator_stabilizer')>
<IBMBackend('simulator_mps')>
<IBMBackend('simulator_extended_stabilizer')>
<IBMBackend('simulator_statevector')>


In [1]:
# Prepare the input circuit.
from qiskit import QuantumCircuit

bell = QuantumCircuit(2)
bell.h(0)
bell.cx(0, 1)
bell.measure_all()

# Execute the circuit
from qiskit_ibm_runtime import SamplerV2 as Sampler
from qiskit_ibm_runtime.fake_provider import FakeManilaV2

# Run the sampler job locally using FakeManilaV2
backend = FakeManilaV2()

from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_bell = pm.run(bell)

sampler = Sampler(backend)

job = sampler.run([(isa_bell,)])
result = job.result()

pub_result = result[0]
# Get counts from the classical register "meas".
print(f" >> Counts for the meas output register: {pub_result.data.meas.get_counts()}")

 >> Counts for the meas output register: {'00': 475, '11': 486, '10': 37, '01': 26}


In [2]:
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

# Create a new circuit with two qubits (first argument) and two classical
# bits (second argument)
qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)

# Set up six different observables.
observables_labels = ["ZZ", "ZI", "IZ", "XX", "XI"]

from qiskit.quantum_info import SparsePauliOp
observables = [SparsePauliOp(label) for label in observables_labels]

# Convert to an ISA circuit and layout-mapped observables.
pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_circuit = pm.run(qc)
observables = [
    observable.apply_layout(isa_circuit.layout) for observable in observables
]

## Molecular Simulations

In [ ]:
from qiskit import Aer
from qiskit.utils import QuantumInstance
from qiskit.algorithms import VQE
from qiskit.circuit.library import TwoLocal
from qiskit.algorithms.optimizers import COBYLA
from qiskit_nature.drivers import PySCFDriver
from qiskit_nature.transformers import FreezeCoreTransformer
from qiskit_nature.converters import QubitConverter
from qiskit_nature.mappers import ParityMapper
from qiskit_nature.algorithms import GroundStateEigensolver

# Define the molecule
driver = PySCFDriver(atom='H .0 .0 .0; H .0 .0 0.735', basis='sto3g')
molecule = driver.run()

# Freeze core orbitals
transformer = FreezeCoreTransformer()
molecule = transformer.transform(molecule)

# Map to qubit Hamiltonian
qubit_converter = QubitConverter(mapper=ParityMapper(), two_qubit_reduction=True)
qubit_hamiltonian = qubit_converter.convert(molecule.second_q_ops()[0])

# Set up the VQE algorithm
optimizer = COBYLA(maxiter=1000)
var_form = TwoLocal(rotation_blocks='ry', entanglement_blocks='cz')
vqe = VQE(var_form, optimizer=optimizer, quantum_instance=QuantumInstance(Aer.get_backend('qasm_simulator')))

# Solve for the ground state energy
solver = GroundStateEigensolver(qubit_converter, vqe)
result = solver.solve(molecule)

print("Ground state energy:", result.total_energies[0])

## Material Design

In [ ]:
from qiskit import Aer
from qiskit.utils import QuantumInstance
from qiskit.algorithms import VQE
from qiskit.circuit.library import TwoLocal
from qiskit.algorithms.optimizers import COBYLA
from qiskit_nature.drivers import PySCFDriver
from qiskit_nature.transformers import FreezeCoreTransformer
from qiskit_nature.converters import QubitConverter
from qiskit_nature.mappers import ParityMapper
from qiskit_nature.algorithms import GroundStateEigensolver

# Define the material (e.g., Lithium Hydride, LiH)
driver = PySCFDriver(atom='Li .0 .0 .0; H .0 .0 1.6', basis='sto3g')
material = driver.run()

# Freeze core orbitals
transformer = FreezeCoreTransformer()
material = transformer.transform(material)

# Map to qubit Hamiltonian
qubit_converter = QubitConverter(mapper=ParityMapper(), two_qubit_reduction=True)
qubit_hamiltonian = qubit_converter.convert(material.second_q_ops()[0])

# Set up the VQE algorithm
optimizer = COBYLA(maxiter=1000)
var_form = TwoLocal(rotation_blocks='ry', entanglement_blocks='cz')
vqe = VQE(var_form, optimizer=optimizer, quantum_instance=QuantumInstance(Aer.get_backend('qasm_simulator')))

# Solve for the ground state energy
solver = GroundStateEigensolver(qubit_converter, vqe)
result = solver.solve(material)

print("Ground state energy:", result.total_energies[0])

#  Shor's Algorithm

In [ ]:
#Purpose: Integer factorization, which has significant implications for cryptography.
from qiskit import QuantumCircuit, Aer, transpile, assemble, execute
from qiskit.algorithms import Shor

# Initialize Shor's algorithm
shor = Shor()

# Factorize a number (e.g., 15)
result = shor.factor(N=15)
print(result)

# Grover's Algorithm

In [ ]:
#Purpose: Unstructured search problems, providing a quadratic speedup over classical algorithms
from qiskit import QuantumCircuit, Aer, transpile, assemble, execute
from qiskit.algorithms import Grover, AmplificationProblem
from qiskit.circuit.library import PhaseOracle

# Define the oracle for the search problem
oracle = PhaseOracle("a & b")

# Define the amplification problem
problem = AmplificationProblem(oracle)

# Initialize Grover's algorithm
grover = Grover()

# Run Grover's algorithm
result = grover.amplify(problem)
print(result)

# Quantum Fourier Transform (QFT)

In [ ]:
#Purpose: A key component in many quantum algorithms, including Shor's algorithm.
from qiskit import QuantumCircuit, Aer, transpile, assemble, execute
from qiskit.circuit.library import QFT

# Create a QFT circuit
qft = QFT(num_qubits=3)

# Simulate the QFT circuit
simulator = Aer.get_backend('qasm_simulator')
result = execute(qft, simulator).result()
print(result.get_counts())

# . Variational Quantum Eigensolver (VQE)


In [ ]:
#Purpose: Finding the ground state energy of a system, useful in quantum chemistry and material science.
from qiskit import Aer
from qiskit.utils import QuantumInstance
from qiskit.algorithms import VQE
from qiskit.circuit.library import TwoLocal
from qiskit.algorithms.optimizers import COBYLA
from qiskit_nature.drivers import PySCFDriver
from qiskit_nature.transformers import FreezeCoreTransformer
from qiskit_nature.converters import QubitConverter
from qiskit_nature.mappers import ParityMapper
from qiskit_nature.algorithms import GroundStateEigensolver

# Define the molecule
driver = PySCFDriver(atom='H .0 .0 .0; H .0 .0 0.735', basis='sto3g')
molecule = driver.run()

# Freeze core orbitals
transformer = FreezeCoreTransformer()
molecule = transformer.transform(molecule)

# Map to qubit Hamiltonian
qubit_converter = QubitConverter(mapper=ParityMapper(), two_qubit_reduction=True)
qubit_hamiltonian = qubit_converter.convert(molecule.second_q_ops()[0])

# Set up the VQE algorithm
optimizer = COBYLA(maxiter=1000)
var_form = TwoLocal(rotation_blocks='ry', entanglement_blocks='cz')
vqe = VQE(var_form, optimizer=optimizer, quantum_instance=QuantumInstance(Aer.get_backend('qasm_simulator')))

# Solve for the ground state energy
solver = GroundStateEigensolver(qubit_converter, vqe)
result = solver.solve(molecule)

print("Ground state energy:", result.total_energies[0])

# Quantum Approximate Optimization Algorithm (QAOA)

In [ ]:
# Purpose: Solving combinatorial optimization problems.
from qiskit import Aer
from qiskit.utils import QuantumInstance
from qiskit.algorithms import QAOA
from qiskit.circuit.library import TwoLocal
from qiskit.algorithms.optimizers import COBYLA
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.algorithms import MinimumEigenOptimizer

# Define the optimization problem
problem = QuadraticProgram()
problem.binary_var('x')
problem.binary_var('y')
problem.minimize(linear={'x': 1, 'y': 2}, quadratic={('x', 'y'): -1})

# Set up the QAOA algorithm
optimizer = COBYLA(maxiter=100)
qaoa = QAOA(optimizer=optimizer, quantum_instance=QuantumInstance(Aer.get_backend('qasm_simulator')))

# Solve the optimization problem
optimizer = MinimumEigenOptimizer(qaoa)
result = optimizer.solve(problem)

print(result)

# Quantum Phase Estimation (QPE)

In [ ]:
# Purpose: Estimating the phase (eigenvalue) of an eigenvector of a unitary operator.
from qiskit import QuantumCircuit, Aer, transpile, assemble, execute
from qiskit.circuit.library import QFT

# Create a QPE circuit
qc = QuantumCircuit(3, 3)
qc.h([0, 1, 2])
qc.cp(1.0, 0, 2)
qc.cp(0.5, 1, 2)
qc.cp(0.25, 2, 2)
qc.append(QFT(3).inverse(), [0, 1, 2])
qc.measure([0, 1, 2], [0, 1, 2])

# Simulate the QPE circuit
simulator = Aer.get_backend('qasm_simulator')
result = execute(qc, simulator).result()
print(result.get_counts())